<center><div class="alert alert-block alert-info" style=" line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px; color:green"> &nbsp; Bengali.AI Speech Recognition : Our very first submission</b><br><br><b style="font-size: 18px; color:green">HELLO EVERYONE!</b><br>
</div></center>

In this Notebook we'll try to submit our first submission by using a publicly available model.

We'll be using this [BanglaASR](https://huggingface.co/bangla-speech-processing/BanglaASR) model. It is actually a finetuned [whisper](https://openai.com/research/whisper) model on rained [Bangla Mozilla Common Voice Dataset](https://arxiv.org/abs/2206.14053). It's reported performance is 4.58% on 7k validation set which seems pretty good to be a starting point. So let's start with that.

I'll be trying to use the other publicly available models one by one and see how they perform. Then we'll get an idea from where we can start the training/finetuning on this huge dataset.

# Imports

In [27]:
import os
import librosa
import torch
import torchaudio
import numpy as np

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
from tqdm import tqdm
import pandas as pd
import soundfile as sf
from pydub import AudioSegment

# Load the model

While submitting notebooks, you need to have internet disabled. So you can't download models from huggingface directly. One workaround is to download the models from huggingface, upload them to kaggle as datasets and then use them!

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "/kaggle/input/bangla-speech-processing-banglaasr/BanglaASR"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

# Demo inference

We'll write an inference function where we'll give the path as input and it will return the transcription!

First we'll need to convert the audio sampling rate to 16k since WhisperFeatureExtractor requires audios to be sampled at 16k.

In [29]:
#This code is modified from the source code provided here https://huggingface.co/bangla-speech-processing/BanglaASR
def inference_fn(path):
    speech_array, sampling_rate = sf.read(mp3_path)
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
    input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(inputs=input_features.to(device))[0]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription


# Experimenting Zone

In [30]:
root_path = "/kaggle/input/narail-1/"
idx = 2
mp3_path = root_path + df['id'].iloc[idx]

print(f"File name :",mp3_path)
AudioSegment.from_file(mp3_path)

File name : /kaggle/input/narail-1/rec_01_audio_10.wav


Looks like it's doing quite good for longer sentences and common words!

# Inference 

In [32]:
mp3_path = "/kaggle/input/narail-1/rec_01_audio_0.wav"
print(f"File name :",mp3_path)
AudioSegment.from_file(mp3_path)

File name : /kaggle/input/narail-1/rec_01_audio_0.wav


In [33]:
import os
test_path = "/kaggle/input/narail-1"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)
    
    #sanity check
    if len(prediction)==0:
        prediction = "<> "
    
    sentences.append(prediction)
    
    

  0%|          | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1000/1000 [57:50<00:00,  3.47s/it] 


Predictions ready. Let's make the csv file

In [34]:
df1 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df1.head()

,External_ID,Contents
0,rec_19_audio_24,সেইডি-এর আনো সোনিসেই আনের বালো জ্যোতিরে তার সে...
1,rec_05_audio_0,দিন এলাকে মসা আসে নেই
2,rec_05_audio_18,"জোর বাতীল দিয়াল ওঠা। এদীগায়, এদীগায়। তবে কল..."
3,rec_08_audio_32,মেল আদির কতাটতা কোয়ানা ইন্ডিয়ায় গেল না ইন্ড...
4,rec_18_audio_10,"তিনি ব্লাস, চার দেবদ্যাল শ্রমান।"


In [35]:
df1

,External_ID,Contents
0,rec_19_audio_24,সেইডি-এর আনো সোনিসেই আনের বালো জ্যোতিরে তার সে...
1,rec_05_audio_0,দিন এলাকে মসা আসে নেই
2,rec_05_audio_18,"জোর বাতীল দিয়াল ওঠা। এদীগায়, এদীগায়। তবে কল..."
3,rec_08_audio_32,মেল আদির কতাটতা কোয়ানা ইন্ডিয়ায় গেল না ইন্ড...
4,rec_18_audio_10,"তিনি ব্লাস, চার দেবদ্যাল শ্রমান।"
...,...,...
995,rec_17_audio_21,"তারফরে ওন্ধে দুই মুট সাইল, উল্লাহ্সাইল, তাই আগ..."
996,rec_09_audio_19,যাকা দ্বান্য লেসসান্যে কেন ক্ষণ ক্ষণ ক্ষণ ক্ষণ...
997,rec_04_audio_6,"এবারা আমারে নাদীদের গোল্লীদের সংস্যাস্ত, এর দু..."
998,rec_16_audio_7,জ্জ্বোয়ারেশীল।্বেরা গেম কর্টা নামায় শেষ।


In [36]:
df1.to_csv("submission1.csv",index=False)

All set. Let's hit the "submit" option and pray!

In [ ]:
========================================================================================================================================

In [38]:
import os
test_path = "/kaggle/input/narail-2"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)
    
    #sanity check
    if len(prediction)==0:
        prediction = "<>"
    
    sentences.append(prediction)
    
    

  0%|          | 0/880 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 880/880 [49:49<00:00,  3.40s/it]


In [39]:
df2 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df2.head()

,External_ID,Contents
0,rec_32_audio_29,"ফিলেজ শহরের জমিন, কমিলা করবান নেকি আমারা তাই করছে"
1,rec_26_audio_43,"যার সেই সময়েলো, আগে, উন্ন ডিগুরি ও নৌ-কায়জাত..."
2,rec_23_audio_76,"তালি, আরেই সবতে, এসবতে, খুবই নোলাল বিশ্ববিদ্যা..."
3,rec_35_audio_36,আমি তেশসব্বিশ তারিখি দেওয়ার। আছে কালকে সন্দেশ...
4,rec_27_audio_3,পরকিলাই নদীহে যেক্ত মাধানিমে করাতে পাইসের মাজে...


In [40]:
df2

,External_ID,Contents
0,rec_32_audio_29,"ফিলেজ শহরের জমিন, কমিলা করবান নেকি আমারা তাই করছে"
1,rec_26_audio_43,"যার সেই সময়েলো, আগে, উন্ন ডিগুরি ও নৌ-কায়জাত..."
2,rec_23_audio_76,"তালি, আরেই সবতে, এসবতে, খুবই নোলাল বিশ্ববিদ্যা..."
3,rec_35_audio_36,আমি তেশসব্বিশ তারিখি দেওয়ার। আছে কালকে সন্দেশ...
4,rec_27_audio_3,পরকিলাই নদীহে যেক্ত মাধানিমে করাতে পাইসের মাজে...
...,...,...
875,rec_29_audio_10,"তাকোর বাইর বিতা ডাকি বুড়ি যাঁড়ে, যা মায়ড়িং..."
876,rec_23_audio_40,"পালায়জাকুয়ারের এরকে মনলের আবিজি, উচ্চ? একটন ..."
877,rec_32_audio_49,"তামরা ত্ব খায় নেই, তামরা খালি খলি সেইজা যাদর ..."
878,rec_26_audio_51,"এরকা দায়া লেখায় বাড়ি এদশাহ হোস্প, লাড়া আতু..."


In [41]:
df2.to_csv("submission2.csv",index=False)

In [ ]:
==================================================================================================================================================

In [42]:
values1 = df1[['External_ID','Contents']]
values2 = df2[['External_ID','Contents']]


dataframes = [values1, values2]

join = pd.concat(dataframes)
join

,External_ID,Contents
0,rec_19_audio_24,সেইডি-এর আনো সোনিসেই আনের বালো জ্যোতিরে তার সে...
1,rec_05_audio_0,দিন এলাকে মসা আসে নেই
2,rec_05_audio_18,"জোর বাতীল দিয়াল ওঠা। এদীগায়, এদীগায়। তবে কল..."
3,rec_08_audio_32,মেল আদির কতাটতা কোয়ানা ইন্ডিয়ায় গেল না ইন্ড...
4,rec_18_audio_10,"তিনি ব্লাস, চার দেবদ্যাল শ্রমান।"
...,...,...
875,rec_29_audio_10,"তাকোর বাইর বিতা ডাকি বুড়ি যাঁড়ে, যা মায়ড়িং..."
876,rec_23_audio_40,"পালায়জাকুয়ারের এরকে মনলের আবিজি, উচ্চ? একটন ..."
877,rec_32_audio_49,"তামরা ত্ব খায় নেই, তামরা খালি খলি সেইজা যাদর ..."
878,rec_26_audio_51,"এরকা দায়া লেখায় বাড়ি এদশাহ হোস্প, লাড়া আতু..."


In [43]:
join.to_csv("3.3-narail-banglaWhisper-inferences.csv")

In [44]:
print('done')

done
